creating a mentors.csv

In [14]:
import pandas as pd

# Define 10 mentors with varied expertise areas and capacities
mentors_data = {
    'mentor_id': ['M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10'],
    'mentor_name': [
        'Dr. Rao', 'Ms. Mehta', 'Mr. Iyer', 'Dr. Khan', 'Ms. Singh', 
        'Prof. Sharma', 'Dr. Kapoor', 'Mr. Joshi', 'Ms. Reddy', 'Dr. Nair'
    ],
    'expertise_area': [
        'Academic', 'Wellness', 'Career', 'Academic', 'Wellness', 
        'Academic', 'Career', 'Wellness', 'Academic', 'Career'
    ],
    'max_capacity': [10, 10, 12, 10, 8, 15, 12, 10, 10, 12],
    'current_load': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # Resetting to zero for fresh allocation
    'availability_hours': [8, 10, 6, 4, 12, 15, 8, 5, 10, 7],
    'intervention_type': [
        'Study Planning', 'Stress Management', 'Career Guidance', 
        'Exam Strategy', 'Emotional Support', 'Research Mentorship', 
        'Job Readiness', 'Mental Health Support', 'Subject Tutoring', 
        'Industry Networking'
    ]
}

# Create the DataFrame
mentors_df = pd.DataFrame(mentors_data)

# Save to CSV
mentors_df.to_csv('mentors.csv', index=False)

print("mentors.csv has been created with 10 mentors.")

mentors.csv has been created with 10 mentors.


In [15]:
students = pd.read_csv("students_updated_SRI.csv")
mentors = pd.read_csv("mentors.csv")

# Verify essential columns
print(students[['student_id', 'APS', 'WWS', 'CRS', 'Category']].head())

  student_id   APS   WWS   CRS Category
0       S001  69.9  83.0  28.2   Yellow
1       S002  80.3  39.5  64.8   Yellow
2       S003  66.7  56.0  73.0   Yellow
3       S004  75.1  65.5  49.0   Yellow
4       S005  84.3  59.5  53.8   Yellow


Classifying the primary need of students

In [16]:
def classify_need(row):
    scores = {
        "Academic": row["APS"],
        "Wellness": row["WWS"],
        "Career": row["CRS"]
    }
    # Primary need is the domain with the lowest score
    return min(scores, key=scores.get)

students["primary_need"] = students.apply(classify_need, axis=1)
students[["student_id", "primary_need"]].head()

,student_id,primary_need
0,S001,Career
1,S002,Wellness
2,S003,Wellness
3,S004,Career
4,S005,Career


Mentor Macthing logic

In [17]:
# Sort to prioritize Red and Yellow students
risk_map = {'Red': 1, 'Yellow': 2, 'Blue': 3, 'Green': 4}
students['risk_priority'] = students['Category'].map(risk_map)
students = students.sort_values(by=['risk_priority', 'SRI'], ascending=[True, True])

students["assigned_mentor"] = "Waitlist"
students["suggested_intervention"] = "Pending"

for index, student in students.iterrows():
    # 1. Look for matching expertise with capacity
    eligible = mentors[
        (mentors["expertise_area"] == student["primary_need"]) &
        (mentors["current_load"] < mentors["max_capacity"])
    ]
    
    # 2. If no specialist, look for any mentor with capacity (Generalist Fallback)
    if eligible.empty:
        eligible = mentors[mentors["current_load"] < mentors["max_capacity"]]
    
    if not eligible.empty:
        # Optimization: Pick mentor with highest availability
        selected = eligible.sort_values(by="availability_hours", ascending=False).iloc[0]
        
        students.at[index, "assigned_mentor"] = selected["mentor_name"]
        students.at[index, "suggested_intervention"] = selected["intervention_type"]
        
        # Update mentor load
        mentors.loc[mentors["mentor_id"] == selected["mentor_id"], "current_load"] += 1

students.head()

,student_id,age,program,semester,gpa,attendance,assignments_completion,stress_levels,sleep_hours,mental_wellbeing,...,APS,WWS,PTMS,CRS,SRI,Category,primary_need,risk_priority,assigned_mentor,suggested_intervention
107,S108,20,MBA,3,4.0,43,56,8,4.2,3,...,44.1,35.0,27.8,26.0,34.130,Red,Career,1,Dr. Kapoor,Job Readiness
103,S104,18,MBA,7,4.1,41,59,9,4.4,3,...,44.6,32.0,33.3,23.2,34.345,Red,Career,1,Dr. Kapoor,Job Readiness
100,S101,18,MBA,5,5.3,57,40,9,4.5,4,...,51.6,34.5,25.0,26.4,35.635,Red,Career,1,Dr. Kapoor,Job Readiness
108,S109,21,MBA,3,4.2,40,40,8,4.5,4,...,41.0,38.5,33.6,27.4,35.805,Red,Career,1,Dr. Kapoor,Job Readiness
104,S105,18,B.Tech,1,4.2,42,47,8,4.6,4,...,43.0,39.0,31.5,26.6,35.845,Red,Career,1,Dr. Kapoor,Job Readiness


Cluster based intervention

In [18]:
def cluster_intervention(category):
    if category == 'Red':
        return "Daily Intensive Support & Faculty Review"
    elif category == 'Yellow':
        return "Bi-weekly Mentorship Sessions"
    elif category == 'Blue':
        return "Monthly Peer Support Group"
    else:
        return "Self-Guided Excellence Workshop"

students["cluster_intervention"] = students["Category"].apply(cluster_intervention)

risk alerts

In [19]:
def generate_alert(row):
    if row["Category"] == "Red":
        return "Immediate Action Required"
    elif row["SRI"] < 50: # Production-level multi-factor check
        return "Critical Multi-Domain Risk"
    else:
        return "No Alert"

students["alert_status"] = students.apply(generate_alert, axis=1)

Final recommendation table

In [20]:
final_table = students[[
    "student_id",
    "Category",
    "SRI",
    "primary_need",
    "assigned_mentor",
    "suggested_intervention",
    "cluster_intervention",
    "alert_status"
]]

final_table.to_csv("final_recommendations.csv", index=False)
print("Production-level recommendations saved.")
final_table.head(10)

Production-level recommendations saved.


,student_id,Category,SRI,primary_need,assigned_mentor,suggested_intervention,cluster_intervention,alert_status
107,S108,Red,34.130,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
103,S104,Red,34.345,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
100,S101,Red,35.635,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
108,S109,Red,35.805,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
104,S105,Red,35.845,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
106,S107,Red,37.125,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
101,S102,Red,37.930,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
109,S110,Red,39.065,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
105,S106,Red,39.465,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
102,S103,Red,40.285,Career,Dr. Kapoor,Job Readiness,Daily Intensive Support & Faculty Review,Immediate Action Required
